In [1]:
import sys, os
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache
from math import ceil, floor

# INPUT

In [2]:
inputfilename = './inputs/day16A.txt'

inputdata = read_txt(inputfilename)

testdata_small = ["###############",
"#.......#....E#",
"#.#.###.#.###.#",
"#.....#.#...#.#",
"#.###.#####.#.#",
"#.#.#.......#.#",
"#.#.#####.###.#",
"#...........#.#",
"###.#.#####.#.#",
"#...#.....#.#.#",
"#.#.#.###.#.#.#",
"#.....#...#.#.#",
"#.###.#.#.#.#.#",
"#S..#.....#...#",
"###############"]

testdata_large = ["#################",
"#...#...#...#..E#",
"#.#.#.#.#.#.#.#.#",
"#.#.#.#...#...#.#",
"#.#.#.#.###.#.#.#",
"#...#.#.#.....#.#",
"#.#.#.#.#.#####.#",
"#.#...#.#.#.....#",
"#.#.#####.#.###.#",
"#.#.#.......#...#",
"#.#.###.#####.###",
"#.#.#...#.....#.#",
"#.#.#.#####.###.#",
"#.#.#.........#.#",
"#.#.#.#########.#",
"#S#.............#",
"#################"]

In [43]:
data_to_use = inputdata

def print_maze_map(maze_map):
    for line in maze_map:
        print(''.join(line))

reindeer_maze = np.array([[x for x in line] for line in data_to_use if '#' in line])

print_maze_map(reindeer_maze)

start_position = tuple(np.array(np.where(reindeer_maze == "S")).flatten())
end_position = tuple(np.array(np.where(reindeer_maze == "E")).flatten())

starting_cost = 0
starting_direction = '>'

start_state = (start_position, starting_direction, starting_cost)

print(start_state, end_position)

#############################################################################################################################################
#...#.......#.............#.........#...................#.............#.....#.....#...........#.....................................#......E#
#.###.###.###.#####.#######.#######.#.#.#####.#########.#####.###.#.#.###.#.###.#.#####.#####.#.###########.###.#.#.###.###.#######.#.#####.#
#...#.#.#.#...#...#.#.......#.....#.#.#.#.....#.......#.#...........#...#.#.....#.#.....#.#...#.....#.....#.#.....................#.#...#.#.#
#.#.#.#.#.#.###.#.#.#.#######.###.#.#.#.###.###.#####.#.#.#.#.###.#####.#.#######.#.#####.#.#########.###.###.#######.#.#.###.###.#.###.#.#.#
#.#.#...#...#...#.#.......#...#.#.#.#.#...#...#...#...#...#.#.........#...#.....#...#...#.#...#.......#.#.....#.......#...#.#.#...#...#...#.#
#.#.###.#####.#.###########.###.#.#.#.###.#######.#.###.#####.###.###.#.###.###.#####.#.#.###.#.#######.#######.#.###.#####.#.#.###.#.###.#.#
#.#...

## PART 1

In [44]:
direction_map = {'>': {'movement':np.array([0, 1]), 'rotations': ['^', 'v']},
                 '<': {'movement':np.array([0, -1]), 'rotations': ['^', 'v']},
                 'v': {'movement':np.array([1, 0]), 'rotations': ['<', '>']},
                 '^': {'movement':np.array([-1, 0]), 'rotations': ['<', '>']}}
advancing_cost = 1
rotating_cost = 1000

def apply_movement(position, direction):
    movement = direction_map[direction]['movement']
    return tuple(position + movement)

def get_next_states(maze_map, state, advance_cost=advancing_cost, rotate_cost=rotating_cost):
    next_states = []
    current_position = state[0]
    current_direction = state[1]
    current_cost = state[2]

    # Advance one step
    next_position = apply_movement(current_position, current_direction)
    if maze_map[next_position] != '#':
        next_states.append((next_position, current_direction, current_cost + advance_cost))

    # Rotate and advance
    for new_direction in direction_map[current_direction]['rotations']:
        next_states.append((current_position, new_direction, current_cost + rotate_cost))

    return next_states

def explore_maze(maze_map, start_state, end_position):
    visited_states = {}
    start_position = start_state[0]
    start_direction = start_state[1]
    start_cost = start_state[2]
    visited_states[start_position] = {}
    visited_states[start_position][start_direction] = start_cost
    states_to_expand = set([start_state])
    while len(states_to_expand) > 0:
        current_state = states_to_expand.pop()

        next_states = get_next_states(maze_map, current_state)
        for next_state in next_states:
            next_position = next_state[0]
            next_direction = next_state[1]
            next_cost = next_state[2]
            if next_position not in visited_states:
                visited_states[next_position] = {}
            if next_direction not in visited_states[next_position]:
                visited_states[next_position][next_direction] = next_cost
                states_to_expand.add(next_state)
            if next_cost < visited_states[next_position][next_direction]:
                visited_states[next_position][next_direction] = next_cost
                states_to_expand.add(next_state)
    return visited_states

In [45]:
visited_states = explore_maze(reindeer_maze, start_state, end_position)
print(visited_states[end_position])

{'^': 111480, '<': 112480, '>': 112480, 'v': 113480}


## PART 2

In [46]:
# REQUIRES THE OUTPUT OF PART 1 TO RUN

reverse_direction_map = {'>': '<',
                '<': '>',
                'v': '^',
                '^': 'v'}
backtracking_cost = -1
backtrack_rotating_cost = -1000

def recover_best_path(visited_states, end_position):
    final_direction = min(visited_states[end_position], key=visited_states[end_position].get)
    backtrack_direction = reverse_direction_map[final_direction]
    final_cost = visited_states[end_position][final_direction]
    
    final_state = (end_position, backtrack_direction, final_cost)

    best_path_tiles = set([end_position])
    states_to_backtrack = set([final_state])

    while len(states_to_backtrack) > 0:
        current_state = states_to_backtrack.pop()
        next_states = get_next_states(reindeer_maze, current_state, backtracking_cost, backtrack_rotating_cost)
        for next_state in next_states:
            next_position = next_state[0]
            next_direction = next_state[1]
            next_cost = next_state[2]
            backtracking_direction = reverse_direction_map[next_direction]
            # Check if this matches the best path - if the cost matches to make it to the end with the optimal cost
            if visited_states[next_position][backtracking_direction] == next_cost:
                best_path_tiles.add(next_position)
                states_to_backtrack.add(next_state)

    return best_path_tiles

In [47]:
len(recover_best_path(visited_states, end_position))

529

In [34]:
visited_states[end_position]

{'>': 8036, '^': 7036, 'v': 9036, '<': 8036}